In [1]:
import os

import numpy as np
import pandas as pd
import scanpy as sc
from sklearn.neural_network import MLPClassifier

import matplotlib.pyplot as plt
import seaborn as sns

sc.settings.n_jobs = 24
sc.settings.set_figure_params(dpi=180, dpi_save=300, frameon=False, figsize=(4, 4), fontsize=8, facecolor='white')

In [2]:
import numpy as np
import pandas as pd
import anndata as ad
from scipy.sparse import csr_matrix
print(ad.__version__)

0.8.0


The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

or the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

In [3]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd

31896


See **functions** in the repository for [AnalysisTool_Chromatin](../../README.md)

In [4]:
# Chromatin_analysis_tools (ATC)
# Get path for the py containing functions
import os
import sys
import importlib
module_path =r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin'
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import relevant modules
import gene_selection 
importlib.reload(gene_selection)
import gene_to_loci
importlib.reload(gene_to_loci)
import gene_activity
importlib.reload(gene_activity)
import loci_1d_features
importlib.reload(loci_1d_features)  

import atac_to_loci
importlib.reload(atac_to_loci)

<module 'atac_to_loci' from 'C:\\Users\\shiwei\\Documents\\AnalysisTool_Chromatin\\atac_to_loci.py'>

In [5]:
%matplotlib inline

sc.settings.set_figure_params(dpi=300, )

# 1. Load celltype codebook with gene annotation

## Codebook

**Codebook** example from [codebook_link](../resources/MERFISH_loci_adjacent_genes_tss_more_res.csv)

In [6]:
# load codebook
# L drive is Crick Pu_SSD_0
codebook_save_folder=r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin\resources'

# Load sorted codebook (allows direct matrix slicing) with cell type info
# Load codebook (then sort to allow direct matrix slicing) with cell type info

codebook_fname = os.path.join(codebook_save_folder,'MERFISH_loci_adjacent_genes_tss_more_res.csv')
codebook_df = pd.read_csv (codebook_fname, index_col=0)

# sort df temporailiy so matrix can be sliced by df order directly
codebook_df = loci_1d_features.sort_loci_df_by_chr_order (codebook_df)

codebook_df.head()

,name,id,chr,chr_order,library,dtype,adjacent_genes_0kb_tss,adjacent_genes_10kb_tss,adjacent_genes_50kb_tss,adjacent_genes_100kb_tss,adjacent_genes_200kb_tss,adjacent_genes_500kb_tss,adjacent_genes_1000kb_tss,adjacent_genes_1500kb_tss,adjacent_genes_2000kb_tss
loci_name,,,,,,,,,,,,,,,
chr1_3742742_3759944,1:3742742-3759944,1,1,0,CTP11,combo,intergenic,intergenic,intergenic,Xkr4,Xkr4,Gm18956; Gm7341; Xkr4,Gm18956; Gm2053; Gm6085; Gm6101; Gm6119; Gm734...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...
chr1_6245958_6258969,1:6245958-6258969,2,1,1,CTP11,combo,intergenic,intergenic,4732440D04Rik; Rb1cc1,4732440D04Rik; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Npbwr1; Rb1cc1...,4732440D04Rik; Gm19026; Gm2147; Gm5694; Gm7182...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...
chr1_8740008_8759916,1:8740008-8759916,3,1,2,CTP11,combo,intergenic,intergenic,Gm15452,Gm15452,Gm15452,Gm15452,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm15452;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...
chr1_9627926_9637875,1:9627926-9637875,1,1,3,CTP13,combo,2610203C22Rik,2610203C22Rik,2610203C22Rik,2610203C22Rik; Adhfe1; Gm6161; Mybl1; Rrs1,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm18300;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Cops5; C...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...
chr1_9799472_9811359,1:9799472-9811359,2,1,4,CTP13,combo,Gm6195,Gm6195; Sgk3,Gm6195; Sgk3,1700034P13Rik; Gm6195; Mcmdc2; Mybl1; Sgk3; Vc...,1700034P13Rik; 2610203C22Rik; Gm6195; Mcmdc2; ...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...


# 2. Load analyzed RNAseq annData

Data can be generated from the notebook (after transfer the cell labels to filtered data):

[external/scripts/sn_rna/2_prepare_and_rename_sn_rna_mop](../../../external/scripts/sn_rna/2_prepare_and_rename_sn_rna_mop.ipynb)

In [7]:
# Get loaded adata from other notebook
import os
import scanpy as sc
# L drive is Crick Pu_SSD_0
scRNA_folder =r'L:\Shiwei\DNA_MERFISH_analysis\10x_nuclei_v3_MOp_AIBS\Analysis_10X_nuclei_v3_AIBS\processed'
# load from here for saved h5ad
adata = sc.read(os.path.join(scRNA_folder,r'MOp_10x_sn_filtered_unnorm_w_label.h5ad'))

In [8]:
#adata_ori = adata.raw.to_adata()
adata_ori = adata

In [9]:
adata_ori.X[:5,:5].toarray()

array([[42.,  9.,  0.,  0.,  0.],
       [14.,  0.,  0.,  0.,  0.],
       [14.,  1.,  0.,  0.,  0.],
       [10.,  0.,  0.,  0.,  0.],
       [12.,  0.,  0.,  0.,  0.]], dtype=float32)

# 3. Extract gene expression 

In [10]:
# change index name as imaged loci to find nearby genes
import gene_to_loci as gl

imaged_loci_df = codebook_df.copy(deep=True)

imaged_loci_df.index.name = 'Imaged_loci'
imaged_loci_df = gl.direct_get_genes_near_gene_dataframe (imaged_loci_df,
                                   codebook_df, 
                                   adjacent_gene_col = None)


imaged_loci_df

Get all existing adjacent gene columns.


,name,id,chr,chr_order,library,dtype,adjacent_genes_0kb_tss,adjacent_genes_10kb_tss,adjacent_genes_50kb_tss,adjacent_genes_100kb_tss,...,adjacent_genes_2000kb_tss,Adjacent_genes_0kb_tss,Adjacent_genes_10kb_tss,Adjacent_genes_50kb_tss,Adjacent_genes_100kb_tss,Adjacent_genes_200kb_tss,Adjacent_genes_500kb_tss,Adjacent_genes_1000kb_tss,Adjacent_genes_1500kb_tss,Adjacent_genes_2000kb_tss
Imaged_loci,,,,,,,,,,,,,,,,,,,,,
chr1_3742742_3759944,1:3742742-3759944,1,1,0,CTP11,combo,intergenic,intergenic,intergenic,Xkr4,...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...,intergenic,intergenic,intergenic,Xkr4,Xkr4,Gm18956; Gm7341; Xkr4,Gm18956; Gm2053; Gm6085; Gm6101; Gm6119; Gm734...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...
chr1_6245958_6258969,1:6245958-6258969,2,1,1,CTP11,combo,intergenic,intergenic,4732440D04Rik; Rb1cc1,4732440D04Rik; Rb1cc1,...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...,intergenic,intergenic,4732440D04Rik; Rb1cc1,4732440D04Rik; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Npbwr1; Rb1cc1...,4732440D04Rik; Gm19026; Gm2147; Gm5694; Gm7182...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...
chr1_8740008_8759916,1:8740008-8759916,3,1,2,CTP11,combo,intergenic,intergenic,Gm15452,Gm15452,...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,intergenic,intergenic,Gm15452,Gm15452,Gm15452,Gm15452,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm15452;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...
chr1_9627926_9637875,1:9627926-9637875,1,1,3,CTP13,combo,2610203C22Rik,2610203C22Rik,2610203C22Rik,2610203C22Rik; Adhfe1; Gm6161; Mybl1; Rrs1,...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...,2610203C22Rik,2610203C22Rik,2610203C22Rik,2610203C22Rik; Adhfe1; Gm6161; Mybl1; Rrs1,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm18300;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Cops5; C...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...
chr1_9799472_9811359,1:9799472-9811359,2,1,4,CTP13,combo,Gm6195,Gm6195; Sgk3,Gm6195; Sgk3,1700034P13Rik; Gm6195; Mcmdc2; Mybl1; Sgk3; Vc...,...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...,Gm6195,Gm6195; Sgk3,Gm6195; Sgk3,1700034P13Rik; Gm6195; Mcmdc2; Mybl1; Sgk3; Vc...,1700034P13Rik; 2610203C22Rik; Gm6195; Mcmdc2; ...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX_166247682_166259932,X:166247682-166259932,1059,X,60,CTP11,combo,intergenic,Gpm6b,Gpm6b,Gemin8; Gpm6b,...,Asb11; Asb9; Bmx; Egfl6; Fancb; Gemin8; Glra2;...,intergenic,Gpm6b,Gpm6b,Gemin8; Gpm6b,Gemin8; Gm15225; Gpm6b; Ofd1; Trappc2,Egfl6; Gemin8; Gm15223; Gm15225; Gm1720; Gpm6b...,Egfl6; Gemin8; Glra2; Gm15223; Gm15225; Gm1523...,Egfl6; Fancb; Gemin8; Glra2; Gm15223; Gm15225;...,Asb11; Asb9; Bmx; Egfl6; Fancb; Gemin8; Glra2;...
chrX_167157164_167167452,X:167157164-167167452,990,X,61,CTP13,combo,intergenic,intergenic,Tmsb4x,Gm15232; Tmsb4x,...,Arhgap6; Egfl6; Frmpd4; Gemin8; Glra2; Gm15223...,intergenic,intergenic,Tmsb4x,Gm15232; Tmsb4x,Gm15232; Gm8814; Tlr7; Tmsb4x,Gm15230; Gm15232; Gm1720; Gm8814; Prps2; Tlr7;...,Egfl6; Gemin8; Gm15223; Gm15230; Gm15232; Gm15...,Egfl6; Frmpd4; Gemin8; Gm15223; Gm15225; Gm152...,Arhgap6; Egfl6; Frmpd4; Gemin8; Glra2; Gm15223...
chrX_168746045_168757590,X:168746045-168757590,1060,X,62,CTP11,combo,intergenic,intergenic,Arhgap6,Arhgap6; Msl3,...,Amelx; Arhgap6; Asmt; Erdr1; Frmpd4; G530011O0...,intergenic,intergenic,Arhgap6,Arhgap6; Msl3,Arhgap6; Frmpd4; Msl3,Amelx; Arhgap6; Frmpd4; Msl3,Amelx; Arhgap6; Frmpd4; Gm15238; Gm15240; Hccs...,Amelx; Arhgap6; Erdr1; Frmpd4; G530011O06Rik; 

## major classes

In [11]:
groupby_adata = 'class_label_new'

np.unique(list(adata_ori.obs[groupby_adata]))
sel_class_to_process = [c for c in np.unique(list(adata_ori.obs[groupby_adata])) if c!='nan']
sel_class_to_process

['Astro', 'Endo', 'GABA', 'Gluta', 'Micro', 'Oligo']

In [12]:
# output_folder
output_folder = r'F:\Chromatin_v3r_result\Gene_expression\10x_snRNA\10x_class'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
    print ('Generate output folder')

In [13]:
%matplotlib inline

import gene_activity
import loci_1d_features
from scipy import stats
import seaborn as sns
from tqdm import tqdm

bin_size =2000 # extend both way, thus equivilant to~ 1Mb
adjcent_col = f'Adjacent_genes_{bin_size}kb_tss'
activity_type = 'sum' # sum of all gene associated to a loci for each single cell

expression_res_df_dict = {}
sel_class_to_process = [c for c in np.unique(list(adata_ori.obs[groupby_adata])) if c!='nan']

for _group in sel_class_to_process[:]:
    
    print (f'Process RNAseq data for {_group}')
    
    expression_res_dict={}
    sorted_group_order = [_group]

    imaged_loci_df_group = loci_1d_features.codebook_chr_order_for_loci_dataframe (imaged_loci_df, 
                                               codebook_df, 
                                               sel_cols =['chr','chr_order','id'], 
                                               sort_df = True,
                                               sort_by_chr=True)

    loci_key_list = loci_1d_features.sorted_loci_keys_for_loci_dataframe(imaged_loci_df_group)
    loci_ori_ind = loci_1d_features.find_chr_loci_iloc_from_loci_keys (codebook_df, loci_key_list)

    # for loci along the chromosome, append the measurements for each single cell
    for _ind, sel_loci_ind in tqdm(enumerate(imaged_loci_df_group.index.tolist()[:])):

        # 4. get adjacent gene (aka 500kb extension both way) expression
        sel_genes=imaged_loci_df_group.loc[sel_loci_ind][f'Adjacent_genes_{bin_size}kb_tss'].split('; ')
        sel_adata =  adata_ori[:,adata_ori.var.index.isin(sel_genes)]
        marker_expressions = gene_activity.gene_activity_raw_groups(sel_genes,
                            sel_adata, 
                            sorted_group_order,
                            groupby_adata,
                            ref_norm_list = [],
                            report_type =activity_type)

        expression_res_dict[_ind]=list(marker_expressions[_group])
        
    # convert dict to df as loci by cell
    expression_res_df = pd.DataFrame.from_dict(expression_res_dict, orient='index')
    expression_res_df['loci_name']=codebook_df.index.tolist()
    expression_res_df = expression_res_df.set_index ('loci_name')
    expression_res_df_dict[_group]=expression_res_df
    # save
    output_df_fname = os.path.join(output_folder, f'MERFISH_loci_10x_expression_2X_{bin_size}kb_for_{_group}.csv')
    expression_res_df.to_csv(output_df_fname)
    print ('=========================================================')


Process RNAseq data for Astro


1982it [01:31, 21.58it/s]


Process RNAseq data for Endo


1982it [01:27, 22.57it/s]


Process RNAseq data for GABA


...

1982it [32:46,  1.01it/s]


Process RNAseq data for Micro


1982it [01:28, 22.39it/s]


Process RNAseq data for Oligo


1982it [01:33, 21.26it/s]


In [14]:
expression_res_df

,0,1,2,3,4,5,6,7,8,9,...,210,211,212,213,214,215,216,217,218,219
loci_name,,,,,,,,,,,,,,,,,,,,,
chr1_3742742_3759944,1.0,1.0,0.0,5.0,5.0,11.0,0.0,1.0,2.0,3.0,...,7.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,3.0
chr1_6245958_6258969,8.0,20.0,0.0,24.0,18.0,7.0,15.0,10.0,11.0,16.0,...,33.0,8.0,16.0,7.0,8.0,8.0,5.0,7.0,4.0,2.0
chr1_8740008_8759916,0.0,5.0,5.0,2.0,8.0,17.0,1.0,2.0,6.0,0.0,...,11.0,3.0,7.0,0.0,6.0,0.0,10.0,0.0,0.0,0.0
chr1_9627926_9637875,0.0,5.0,6.0,2.0,7.0,18.0,1.0,2.0,2.0,0.0,...,11.0,2.0,4.0,0.0,5.0,0.0,5.0,0.0,0.0,1.0
chr1_9799472_9811359,0.0,5.0,6.0,2.0,7.0,18.0,1.0,2.0,2.0,0.0,...,11.0,2.0,4.0,0.0,5.0,0.0,5.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX_166247682_166259932,0.0,4.0,5.0,4.0,4.0,2.0,5.0,9.0,2.0,10.0,...,13.0,14.0,1.0,10.0,0.0,8.0,1.0,5.0,0.0,2.0
chrX_167157164_167167452,0.0,4.0,5.0,4.0,3.0,9.0,5.0,9.0,2.0,9.0,...,14.0,14.0,4.0,10.0,3.0,8.0,3.0,5.0,0.0,8.0
chrX_168746045_168757590,0.0,0.0,3.0,0.0,0.0,7.0,1.0,0.0,0.0,0.0,...,6.0,1.0,3.0,1.0,3.0,4.0,3.0,0.0,0.0,7.0


## subclasses

In [15]:
groupby_adata = 'subclass_label_new'

np.unique(list(adata_ori.obs[groupby_adata]))

array(['Astro', 'Endo', 'L2/3 IT', 'L4/5 IT', 'L5 ET', 'L5 IT', 'L5/6 NP',
       'L6 CT', 'L6 IT', 'L6b', 'Lamp5', 'Micro', 'OPC', 'Oligo', 'Pvalb',
       'Sncg', 'Sst', 'Vip', 'nan'], dtype='<U32')

In [16]:
# output_folder
output_folder = r'F:\Chromatin_v3r_result\Gene_expression\10x_snRNA\10x_subclass'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
print ('Generate output folder')

Generate output folder


In [17]:
%matplotlib inline

import gene_activity
import loci_1d_features
from scipy import stats
import seaborn as sns
from tqdm import tqdm

bin_size =2000 # extend both way, thus equivilant to~ 1Mb
adjcent_col = f'Adjacent_genes_{bin_size}kb_tss'
activity_type = 'sum' # sum of all gene associated to a loci for each single cell

expression_res_df_dict = {}

expression_res_df_dict = {}
sel_class_to_process = [c for c in np.unique(list(adata_ori.obs[groupby_adata])) if c!='nan']

for _group in sel_class_to_process[:]:
    
    print (f'Process RNAseq data for {_group}')
    
    expression_res_dict={}
    sorted_group_order = [_group]
    
    _group_fname = _group.replace('/','_').replace(' ','_')

    imaged_loci_df_group = loci_1d_features.codebook_chr_order_for_loci_dataframe (imaged_loci_df, 
                                               codebook_df, 
                                               sel_cols =['chr','chr_order','id'], 
                                               sort_df = True,
                                               sort_by_chr=True)

    loci_key_list = loci_1d_features.sorted_loci_keys_for_loci_dataframe(imaged_loci_df_group)
    loci_ori_ind = loci_1d_features.find_chr_loci_iloc_from_loci_keys (codebook_df, loci_key_list)

    # for loci along the chromosome, append the measurements for each single cell
    for _ind, sel_loci_ind in tqdm(enumerate(imaged_loci_df_group.index.tolist()[:])):

        # 4. get adjacent gene (aka 500kb extension both way) expression
        sel_genes=imaged_loci_df_group.loc[sel_loci_ind][f'Adjacent_genes_{bin_size}kb_tss'].split('; ')
        sel_adata =  adata_ori[:,adata_ori.var.index.isin(sel_genes)]
        marker_expressions = gene_activity.gene_activity_raw_groups(sel_genes,
                            sel_adata, 
                            sorted_group_order,
                            groupby_adata,
                            ref_norm_list = [],
                            report_type =activity_type)

        expression_res_dict[_ind]=list(marker_expressions[_group])
        
    # convert dict to df as loci by cell
    expression_res_df = pd.DataFrame.from_dict(expression_res_dict, orient='index')
    expression_res_df['loci_name']=codebook_df.index.tolist()
    expression_res_df = expression_res_df.set_index ('loci_name')
    expression_res_df_dict[_group]=expression_res_df
    # save
    output_df_fname = os.path.join(output_folder, f'MERFISH_loci_10x_expression_2X_{bin_size}kb_for_{_group_fname}.csv')
    expression_res_df.to_csv(output_df_fname)
    print ('=========================================================')


Process RNAseq data for Astro


1982it [01:32, 21.35it/s]


Process RNAseq data for Endo


1982it [01:29, 22.25it/s]


Process RNAseq data for L2/3 IT


...

1982it [01:39, 19.97it/s]


Process RNAseq data for Sst


1982it [02:55, 11.27it/s]


Process RNAseq data for Vip


1982it [02:13, 14.82it/s]


In [18]:
expression_res_df

,0,1,2,3,4,5,6,7,8,9,...,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091
loci_name,,,,,,,,,,,,,,,,,,,,,
chr1_3742742_3759944,16.0,24.0,14.0,17.0,9.0,13.0,8.0,15.0,14.0,7.0,...,29.0,23.0,35.0,28.0,22.0,5.0,12.0,25.0,37.0,15.0
chr1_6245958_6258969,9.0,6.0,16.0,7.0,7.0,2.0,5.0,9.0,2.0,4.0,...,0.0,6.0,6.0,6.0,9.0,6.0,5.0,0.0,6.0,0.0
chr1_8740008_8759916,40.0,7.0,9.0,35.0,51.0,42.0,12.0,11.0,29.0,5.0,...,24.0,24.0,77.0,88.0,32.0,57.0,52.0,26.0,24.0,51.0
chr1_9627926_9637875,46.0,6.0,18.0,39.0,56.0,53.0,19.0,17.0,29.0,8.0,...,28.0,21.0,129.0,109.0,58.0,60.0,57.0,31.0,22.0,58.0
chr1_9799472_9811359,46.0,6.0,18.0,39.0,56.0,53.0,19.0,17.0,29.0,8.0,...,28.0,21.0,129.0,109.0,58.0,60.0,57.0,31.0,22.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX_166247682_166259932,7.0,4.0,6.0,0.0,0.0,4.0,1.0,4.0,2.0,1.0,...,5.0,6.0,8.0,0.0,12.0,5.0,2.0,1.0,3.0,4.0
chrX_167157164_167167452,46.0,22.0,36.0,17.0,22.0,14.0,114.0,74.0,9.0,10.0,...,33.0,81.0,97.0,19.0,44.0,5.0,9.0,19.0,9.0,24.0
chrX_168746045_168757590,42.0,18.0,32.0,20.0,23.0,10.0,113.0,70.0,11.0,10.0,...,32.0,75.0,94.0,19.0,36.0,0.0,8.0,19.0,7.0,24.0
